### Declare libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
# evaluate an xgboost regression model on the housing dataset
import numpy as np
import pandas as pd
# libraries for visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(style="white")
import os
# library to save the money
import pickle

### Set folder location of the data and load data 

### Declare functions and select the variables that i am going to use

In [2]:
# función creada por mi para traer los NA's
def get_na(df):
    qsna=df.shape[0]-df.isnull().sum(axis=0)
    qna=df.isnull().sum(axis=0)
    ppna=round(100*(df.isnull().sum(axis=0)/df.shape[0]),2)
    aux= {'datos sin NAs en q': qsna, 'Na en q': qna ,'Na en %': ppna}
    na=pd.DataFrame(data=aux)
    return na.sort_values(by='Na en %',ascending=False)

In [3]:
# you need to change it if you are going to run it on your local machine, writing the folder where the raw data of the challenge is
os.chdir('/Users/iairlinker/Documents/repos/cornershop_test/data')
# Loading pre-processed dataset
model_data = pd.read_csv('interim/results.csv', sep=';')
model_data['is_weighted'] = model_data.KG.apply(lambda x: 1 if x>0 else 0)
model_data['on_demand'] = model_data.on_demand.apply(lambda x: 1 if x>0 else 0)
col_names = ['on_demand', 'is_weighted','UN','picking_speed', 
             'seniority','order_stores_distance',
             'total_minutes',
            ]                    
X = model_data[col_names]
y = X.total_minutes
X.drop('total_minutes', axis=1, inplace=True)
X = pd.get_dummies(X)
get_na(X)

,datos sin NAs en q,Na en q,Na en %
UN,1995,5,0.25
on_demand,2000,0,0.00
is_weighted,2000,0,0.00
picking_speed,2000,0,0.00
order_stores_distance,2000,0,0.00
seniority_ADVANCED,2000,0,0.00
seniority_BEGINNER,2000,0,0.00
seniority_INTERMEDIATE,2000,0,0.00
seniority_REVIEW,2000,0,0.00


In [9]:
file_name = "processed/xgb_reg.pkl"
# load the model
model_loaded = pickle.load(open(file_name, "rb"))
# predict
model_data['total_minutes'] = model_loaded.predict(X)
# save the file
model_data[['order_id','total_minutes']].to_csv('processed/predictions.csv', sep=',',index=False)